In [1]:
import tensorflow as tf
import datetime

In [2]:
(train_image, train_labels), (test_image, test_labels) = tf.keras.datasets.mnist.load_data()

In [3]:
train_image = tf.expand_dims(train_image, -1)
test_image = tf.expand_dims(test_image, -1)

train_image = tf.cast(train_image / 255, tf.float32)
test_image = tf.cast(test_image / 255, tf.float32)

train_labels = tf.cast(train_labels, tf.int64)
test_labels = tf.cast(test_labels, tf.int64)

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((train_image, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_image, test_labels))

In [5]:
dataset = dataset.repeat().shuffle(60000).batch(128)

In [6]:
test_dataset = test_dataset.repeat().batch(128)

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3, 3], activation='relu', input_shape = (None, None, 1)),
    tf.keras.layers.Conv2D(32, [3, 3], activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [8]:
'''
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy']
)
'''

"\nmodel.compile(optimizer = 'adam',\n              loss = 'sparse_categorical_crossentropy',\n              metrics = ['accuracy']\n)\n"

# 用tensorboard可视化，用callback方法回调

In [9]:
import os

In [10]:
log_dir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [11]:
#log_dir='logs' : 放在当前目录下logs文件下
#histogram_freq = 1 ：记录直方图的频率，1为每个epoch记录一次。
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq = 1)

In [12]:

file_writer = tf.summary.create_file_writer(log_dir + '/learning_rate')


In [13]:

file_writer.set_as_default()


In [14]:
'''
def lr_sche(epoch):
    learning_rate = 0.2
    if(epoch > 5 and epoch <= 10):
        learning_rate = 0.02
    elif(epoch > 10 and epoch <= 20):
        learning_rate = 0.01
    elif(epoch > 20 and epoch <= 30):
        learning_rate = 0.005
    tf.summary.scalar('learning_rate', data = learning_rate, step = epoch)
    return learning_rate
    '''

"\ndef lr_sche(epoch):\n    learning_rate = 0.2\n    if(epoch > 5 and epoch <= 10):\n        learning_rate = 0.02\n    elif(epoch > 10 and epoch <= 20):\n        learning_rate = 0.01\n    elif(epoch > 20 and epoch <= 30):\n        learning_rate = 0.005\n    tf.summary.scalar('learning_rate', data = learning_rate, step = epoch)\n    return learning_rate\n    "

In [15]:
'''
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_sche)
'''

'\nlr_callback = tf.keras.callbacks.LearningRateScheduler(lr_sche)\n'

In [16]:
'''
model.fit(dataset,
          epochs = 25,
          steps_per_epoch = 60000 // 128,
          validation_data = test_dataset,
          validation_steps = 10000 // 128,
          callbacks = [tensorboard_callback, lr_callback]
)
'''

'\nmodel.fit(dataset,\n          epochs = 25,\n          steps_per_epoch = 60000 // 128,\n          validation_data = test_dataset,\n          validation_steps = 10000 // 128,\n          callbacks = [tensorboard_callback, lr_callback]\n)\n'

In [17]:
'''
%load_ext tensorboard
%matplotlib inline
'''

'\n%load_ext tensorboard\n%matplotlib inline\n'

In [18]:
'''
%tensorboard --logdir logs
'''

'\n%tensorboard --logdir logs\n'

# 自定义训练中使用Tensorboard

In [19]:
optimizer = tf.keras.optimizers.Adam()

In [20]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()

In [21]:
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)

In [22]:
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [23]:
def train_step(model, images, labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels, pred)
    grads = t.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(loss_step)
    train_accuracy(labels, pred)

In [24]:
def test_step(model, images, labels):
    pred = model(images)
    loss_step = loss_func(labels, pred)
    test_loss(loss_step)
    test_accuracy(labels, pred)

In [25]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [26]:
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [27]:
def train():
    for epoch in range(10):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(images, labels)
        with train_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
        print('Epoch{} is finished, loss = {}, acc = {}'.format(epoch, 
                                                                train_loss.result(),
                                                                train_acc.result())) 
        for (batch, (images, labels)) in enumerate(test_dataset):
            test_step(images, labels)                                                        
        print('                     test_loss = {}, test_acc = {}'.format(
                                                     test_loss.result(),
                                                     test_acc.result()))
        train_loss.reset_states()
        train_acc.reset_states()
        test_loss.reset_states()
        test_acc.reset_states()

In [28]:
train()

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

.................................................................................................................................................................................................................................................................................................................................................................................................................................................................

KeyboardInterrupt: 

In [ ]:
%tensorboard --logdir logs/gradient_tape